In [1]:
pip install yahoofinancials

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 KB 722.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 4.4 MB/s eta 0:00:00
  Created wheel for yahoofinancials: filename=yahoofinancials-1.14-py3-none-any.whl size=28645 sha256=bcccf66d1f7991d0ff50b48fa3ac1a97a62fb7675cd21d7e8c6118f3a6193666
  Stored in directory: /root/.cache/pip/wheels/87/45/f7/fd649fd9438ee08551c764771ef1c7358b6a318a65b75ac8e9
Successfully built yahoofinancials
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1


In [2]:
from yahoofinancials import YahooFinancials
from pandas.core.window.rolling import concat
import pandas as pd

import plotly.express as px
import math
from numpy import nan

In [5]:

def extract_data_yearly(ticker):
  tck = YahooFinancials(ticker)
  all_statements_yr =  tck.get_financial_stmts('annual', ['income', 'cash', 'balance'])

  data_is_yr=all_statements_yr['incomeStatementHistory'][ticker]
  data_cf_yr=all_statements_yr['cashflowStatementHistory'][ticker]
  data_bs_yr=all_statements_yr['balanceSheetHistory'][ticker]

  df_is_yr=pd.DataFrame()
  for d in data_is_yr:
    date = list(d.keys())[0]
    vals = d[date]
    vals['Date'] = date
    df_is_yr = df_is_yr.append(pd.DataFrame(vals, index=[0]))

  df_cf_yr=pd.DataFrame()
  for d in data_cf_yr:
    date = list(d.keys())[0]
    vals = d[date]
    vals['Date'] = date
    df_cf_yr = df_cf_yr.append(pd.DataFrame(vals, index=[0]))

  df_bs_yr=pd.DataFrame()
  for d in data_bs_yr:
    date = list(d.keys())[0]
    vals = d[date]
    vals['Date'] = date
    df_bs_yr = df_bs_yr.append(pd.DataFrame(vals, index=[0]))

  merge1=pd.merge(df_is_yr, df_cf_yr, on='Date')
  merge2=pd.merge(merge1, df_bs_yr, on='Date')

  merge2['year'] = merge2['Date'].str[:4]
  merge2['ticker'] = ticker

  return merge2


def extract_data_quarterly(ticker):
  tck = YahooFinancials(ticker)
  all_statements_qt =  tck.get_financial_stmts('quarterly', ['income', 'cash', 'balance'])

  data_is_qt=all_statements_qt['incomeStatementHistoryQuarterly'][ticker]
  data_cf_qt=all_statements_qt['cashflowStatementHistoryQuarterly'][ticker]
  data_bs_qt=all_statements_qt['balanceSheetHistoryQuarterly'][ticker]

  df_is_qt=pd.DataFrame()
  df_cf_qt=pd.DataFrame()
  df_bs_qt=pd.DataFrame()



def ratios_yearly(ticker_):
  df=pd.DataFrame()
  dfc=extract_data_yearly(ticker_)
  dfc['ebitda'] = dfc['operatingIncome'] + dfc['depreciationAndAmortization'] 
  dfc['EnterpriseValue'] = dfc['stockholdersEquity'] + dfc['totalDebt'] - dfc['cashAndCashEquivalents']

  df['ebit_to_interestExpense'] = dfc['ebit'].divide(abs(dfc['interestExpense']))
  df['ebitda_to_interestExpense'] = dfc['ebitda'].divide(abs(dfc['interestExpense']))
  df['FreeOperCF_to_TotalDebt'] = (dfc['ebitda'] - dfc['capitalExpenditure']).divide(dfc['totalDebt'])
  df['FFO_to_TotalDebt'] = (dfc['ebitda'] + dfc['saleOfPPE']).divide(dfc['totalDebt'])
  df['Return_on_Capital'] = (dfc['operatingIncome'] - dfc['deferredIncomeTax']).divide(dfc['EnterpriseValue'])
  df['ebitda_to_Revenue'] = dfc['ebitda'].divide(dfc['totalRevenue'])
  df['TotalDebt_to_EnterpriseValue'] = dfc['totalDebt'].divide(dfc['EnterpriseValue'])
  df['TotalDebt_to_ebitda'] = dfc['totalDebt'].divide(dfc['ebitda'])
  df['ROCE'] = dfc['ebit'].divide(dfc['totalAssets'] + dfc['currentLiabilities'])
  #df['ROIC'] = (dfc['netIncome']-dfc['dividendsPaid']).divide(dfc['TotalDebt'] + dfc['totalStockholderEquity'])

  df['DIO'] = (((dfc['inventory'].add(dfc['inventory'].shift())).div(2)).divide(dfc['costOfRevenue'])).mul(365)
  df['DPO'] = (((dfc['accountsPayable'].add(dfc['accountsPayable'].shift())).div(2)).divide(dfc['costOfRevenue'])).mul(365)
  df['DSO'] = (((dfc['accountsReceivable'].add(dfc['accountsReceivable'].shift())).div(2)).divide(dfc['totalRevenue'])).mul(365)
  df['InventoryTurnover'] = dfc['costOfRevenue'].div(((dfc['inventory'].add(dfc['inventory'].shift())).div(2)))
  df['CCC'] = df['DIO'] + df['DSO'] - df['DPO']

  df['year']=dfc['year']

  df['ticker']=ticker_

  return df

In [17]:
lst=extract_data_yearly('CAT').keys()
for i in lst:
  print(i)


ticker_test='CAT'

financial_data=extract_data_yearly(ticker_test)



ratios_data=ratios_yearly(ticker_test)

netInterestIncome
gainOnSaleOfSecurity
eBITDA
minorityInterests
normalizedIncome
totalExpenses
netNonOperatingInterestIncomeExpense
dilutedEPS
otherIncomeExpense
operatingIncome
grossProfit
netIncomeCommonStockholders
reconciledCostOfRevenue
basicAverageShares
ebit
specialIncomeCharges
netIncome_x
taxRateForCalcs
interestExpense
dilutedNIAvailtoComStockholders
pretaxIncome
earningsFromEquityInterestNetOfTax
totalUnusualItems
interestExpenseNonOperating
totalRevenue
impairmentOfCapitalAssets
basicEPS
costOfRevenue
totalUnusualItemsExcludingGoodwill
netIncomeFromContinuingAndDiscontinuedOperation
taxProvision
netIncomeFromContinuingOperationNetMinorityInterest
reconciledDepreciation
otherNonOperatingIncomeExpenses
sellingGeneralAndAdministration
operatingExpense
netIncomeContinuousOperations
operatingRevenue
totalOperatingIncomeAsReported
normalizedEBITDA
taxEffectOfUnusualItems
netIncomeIncludingNoncontrollingInterests
otherOperatingExpenses
researchAndDevelopment
dilutedAverageShares
D

In [24]:
def plot_style(paper_bg='rgba(0,0,0,0)', plot_bg='#e8ecfc', font='#0F1026'):
    plot_style = {'paper_bgcolor':paper_bg, 'plot_bgcolor':plot_bg,
                               'font': {'color':font}}
    return plot_style

my_palette1 ={'ebit_to_interestExpense': '#FFA41B', 'ebitda_to_interestExpense': '#4C71DD'}



def update_tcc_bar3(ticks, Lambda):
    tickers_list=ticks    
    
    df0 = extract_data_yearly(tickers_list[0])
    
    df1 = df0.copy()
    
    df2 = df0.copy()
    
    df0['TCC'] = 'Without TCC'
    
    df1['TCC'] = 'With TCC'
    
    df1['TotalDebt'] = df0['totalDebt'].mul(1-Lambda)
    df1['interestExpense'] = df0['interestExpense'].mul(1-Lambda)
    df1['operatingIncome'] = df0['operatingIncome'] - abs(df0['interestExpense']).mul(Lambda)
    #df1['ebitda'] = df0['ebitda'] - abs(df0['interestExpense']).mul(Lambda)
    
    df2['totalDebt'] = df0['totalDebt'].mul(1-0.8)
    df2['interestExpense'] = df0['interestExpense'].mul(1-0.8)
    df2['operatingIncome'] = df0['operatingIncome'] - abs(df0['interestExpense']).mul(0.8)
    #df2['ebitda'] = df0['ebitda'] - abs(df0['interestExpense']).mul(0.8)
    
    df = pd.concat([df0, df1], axis=0)
    
    df['Date'] = pd.to_datetime(df['Date'])
    
    df_ratios = df[['ticker', 'Date', 'TCC']]
    df_ratios2 = df[['ticker', 'Date']]
    
    #df = df[['ticker', 'date', 'interestExpense', 'TotalDebt', 'ebitda', 'operatingIncome']]
    
    df_ratios['ebit_to_interestExpense'] = df['ebit'].divide(abs(df['interestExpense']))
    df_ratios['ebitda_to_interestExpense'] = (df['operatingIncome'] 
                                              + df['depreciationAndAmortization']).divide(abs(df['interestExpense']))

    
    df_ratios2['ebit_to_interestExpense'] = df2['ebit'].divide(abs(df2['interestExpense']))
    df_ratios2['ebitda_to_interestExpense'] = (df2['operatingIncome'] 
                                              + df2['depreciationAndAmortization']).divide(abs(df2['interestExpense']))

    
    tcc3_max = max(df_ratios2['ebit_to_interestExpense'].max(),
                   df_ratios2['ebitda_to_interestExpense'].max())
    
    ratios_melt = pd.melt(df_ratios, id_vars=['ticker', 'Date', 'TCC'])
                                                          
    ratios_melt['Date'] = pd.to_datetime(ratios_melt['Date'])
    
    ratios_melt_cols3 = ratios_melt[ratios_melt.variable.isin(['ebit_to_interestExpense', 
                                                'ebitda_to_interestExpense'])]
    
    #ratios_melt_cols3=display_formating(ratios_melt_cols3)
    ratios_melt_cols3['Date_formated']=ratios_melt_cols3['Date'].astype(str).str[0:4]

    color_discrete_map=my_palette1
    
    if (Lambda < 0.8):
        tcc3_bar = px.bar(ratios_melt_cols3, x="Date_formated", y="value", facet_row='ticker',
                          facet_col='TCC',
                          labels={
                              "value":"Ratio",
                              "variable":" ",
                              "Date_formated": ""
                         },
                          color="variable", color_discrete_map= color_discrete_map,
                          barmode="group", range_y=[-0.1*tcc3_max, 1.1*tcc3_max],text_auto='.2s')
    else:    
        #color_discrete_sequence[5] = '#609cd4'
        tcc3_bar = px.bar(ratios_melt_cols3, x="Date_formated", y="value", facet_row='ticker',
                          facet_col='TCC',
                          labels={
                              "value":"Ratio",
                              "variable":" ",
                              "Date_formated": ""
                         },
                          color='variable', color_discrete_map= color_discrete_map,
                          barmode="group", text_auto='.2s')
    
    tcc3_bar.update_layout(plot_style())
    tcc3_bar.update_layout(
        legend=dict(
            x=0,
            y=1,
            traceorder="reversed",
            bgcolor="#ffffff",
            bordercolor="Black",
            borderwidth=2,
            title=None,
        )
    )
                         
    tcc3_bar.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    tcc3_bar.update_annotations(font_size=25)
    
    return tcc3_bar

def update_ratios2(ticks):    
    
    ratios = ratios_yearly(ticks[0])
    
    ratios['Date_formated']=ratios['year']
    
    tickers_list = ticks
    
    #ratios1 = ratios.drop(['date2'], axis=1)

    ratios_melt = pd.melt(ratios, id_vars=['ticker','Date_formated'])

    #ratios_melt['date2'] = pd.to_datetime(ratios_melt['date2'])

    ratios_melt_cols = ratios_melt[ratios_melt.variable.isin(['InventoryTurnover','CCC'])]

    ratios_melt_cols_ticks = ratios_melt_cols[ratios_melt_cols.ticker.isin(tickers_list)].dropna()
        
    ratiosfig = px.bar(ratios_melt_cols_ticks, x="Date_formated", y="value",
                         color="variable", 
                        labels={
                                          "value":"Number of days",
                                          "variable":" ",
                                          "Date_formated": ""
                                     },
                         barmode="group", facet_row="ticker",text_auto='.0f',text="variable")
    #ratiosfig.update_layout(plot_style())
    
    ratiosfig.update_xaxes(tickvals = list(ratios_melt_cols_ticks['Date_formated']))
    
    ratiosfig.update_layout(xaxis = dict(tickfont = dict(size=25)))
    
    ratiosfig.update_layout(font=dict(size=12))
    
    newnames = {'InventoryTurnover': 'Inventory Turnover',
                'CCC': 'Cash Convertion Cycle'}
    ratiosfig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                          legendgroup = newnames[t.name],
                                          hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                         )
                      )

    ratiosfig.update_layout(legend=dict(
        orientation="h",
        title= None,
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1,
        font=dict(
            size=13,
            color="black"
        ),
    ))
 
    return ratiosfig


def bar2_compare(ticks, Lambda):

    my_palette2 ={'FreeOperCF_to_TotalDebt': '#FFA41B', 'FFO_to_TotalDebt': '#4C71DD'}

    tickers_list=ticks    
    
    df0 = extract_data_yearly(tickers_list[0])
    
    df1 = df0.copy()
    
    df2 = df0.copy()
    
    df0['TCC'] = 'Without TCC'
    
    df1['TCC'] = 'With TCC'
    
    df0['ebitda'] = df0['operatingIncome'] + df0['depreciationAndAmortization']    
    #df1['ebitda'] = df1['operatingIncome'] + df1['depreciation']
    #df2['ebitda'] = df2['operatingIncome'] + df2['depreciation']

    df1['totalDebt'] = df0['totalDebt'].mul(1 - Lambda)
    df1['interestExpense'] = df0['interestExpense'].mul(1 - Lambda)
    df1['operatingIncome'] = df0['operatingIncome'] - abs(df0['interestExpense']).mul(Lambda)
    df1['ebitda'] = df0['ebitda'] - abs(df0['interestExpense']).mul(Lambda)

    df2['totalDebt'] = df0['totalDebt'].mul(1 - 0.8)
    df2['interestExpense'] = df0['interestExpense'].mul(1 - 0.8)
    df2['operatingIncome'] = df0['operatingIncome'] - abs(df0['interestExpense']).mul(0.8)
    df2['ebitda'] = df0['ebitda'] - abs(df0['interestExpense']).mul(0.8)

    df = pd.concat([df0, df1], axis=0)

    df['Date'] = pd.to_datetime(df['Date'])

    df_ratios = df[['ticker', 'Date', 'TCC']]
    df_ratios2 = df[['ticker', 'Date']]

    # df = df[['ticker', 'date', 'interestExpense', 'TotalDebt', 'ebitda', 'operatingIncome']]

    df_ratios['FreeOperCF_to_TotalDebt'] = (df['operatingIncome'] + df['depreciationAndAmortization']
                                            - df['capitalExpenditure']).divide(df['totalDebt'])
    df_ratios['FFO_to_TotalDebt'] = (df['operatingIncome']
                                     + df['depreciationAndAmortization'] + df['saleOfPPE']).divide(df['totalDebt'])

    df_ratios2['FreeOperCF_to_TotalDebt'] = (df2['operatingIncome'] + df2['depreciationAndAmortization']
                                             - df2['capitalExpenditure']).divide(df2['totalDebt'])
    df_ratios2['FFO_to_TotalDebt'] = (df2['operatingIncome']
                                      + df2['depreciationAndAmortization'] + df2['saleOfPPE']).divide(df2['totalDebt'])

    tcc4_max = max(df_ratios2['FreeOperCF_to_TotalDebt'].max(),
                   df_ratios2['FFO_to_TotalDebt'].max())

    ratios_melt = pd.melt(df_ratios, id_vars=['ticker', 'Date', 'TCC'])

    ratios_melt['Date'] = pd.to_datetime(ratios_melt['Date'])

    ratios_melt_cols4 = ratios_melt[ratios_melt.variable.isin(['FreeOperCF_to_TotalDebt', 'FFO_to_TotalDebt'])]

    color_discrete_map = my_palette2
    
    ratios_melt_cols4['Date_formated']=ratios_melt_cols4['Date'].astype(str).str[0:4]

    if (Lambda < 0.8):
        fig7 = px.bar(ratios_melt_cols4, x="Date_formated", y="value", facet_row='ticker',
                          facet_col='TCC', 
                          labels={
                                  "value":"Ratio",
                                  "variable":" ",
                                  "Date_formated": ""
                             },
                          color="variable", range_y=[-0.1 * tcc4_max, 1.1 * tcc4_max],
                          color_discrete_map=color_discrete_map, barmode="group",text_auto='.2f')
    else:
        fig7 = px.bar(ratios_melt_cols4, x="Date_formated", y="value", facet_row='ticker',
                          facet_col='TCC', 
                          labels={
                                  "value":"Ratio",
                                  "variable":" ",
                                  "Date_formated": ""
                             },
                          color="variable", color_discrete_map=color_discrete_map, barmode="group",text_auto='.2f')

    fig7.update_layout(plot_style())
    fig7.update_layout(
        legend=dict(
            x=0,
            y=1,
            traceorder="reversed",
            bgcolor="#ffffff",
            bordercolor="Black",
            borderwidth=2,
            title=None,
        )
    )
                         
    fig7.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    fig7.update_annotations(font_size=25)

    return fig7

In [25]:
#update_tcc_bar3(['CAT'],0.1).show()
bar2_compare(['CAT'],0.1).show()

<ipython-input-24-cd2ce2034dc7>:204: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-24-cd2ce2034dc7>:206: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-24-cd2ce2034dc7>:209: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-24-c